In [1]:
import pandas as pd
df = pd.read_csv('/data_vault/COmparative_Study_of_Multimodal_Represenations/data/processed/amazon/train.csv')
df1 = pd.read_csv('/data_vault/COmparative_Study_of_Multimodal_Represenations/data/processed/amazon/test.csv')
df2= pd.read_csv('/data_vault/COmparative_Study_of_Multimodal_Represenations/data/processed/amazon/val.csv')

print(len(df), len(df1), len(df2))

3240000 400000 360000
